In [1]:
#Import knižníc
import pandas as pd
import re
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from transformers.optimization import AdamW
from sklearn.preprocessing import LabelEncoder

/Users/annahavryliak/IdeaProjects/ZS2/pythonProject/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/annahavryliak/IdeaProjects/ZS2/pythonProject/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Stiahnutie dataset
train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/val.csv')

def clean_text(text):
    #Previesť text na malé písmená
    text = text.lower()
    
    #Odstránenie adries URL
    text = re.sub(r'http\S+', '', text)
    
    #Odstránenie hashtagov (iba symbol „#“, text ponechajte)
    text = re.sub(r'#', '', text)
    
    #Odstránenie špeciálnych znakov a čísel
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    
    #Odstránenie zbytočných medzier
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

train_data['cleaned_tweet'] = train_data['tweet'].apply(clean_text)
val_data['cleaned_tweet'] = val_data['tweet'].apply(clean_text)

In [3]:
#Definovanie hyperparametrov
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 2e-5

In [4]:
#Stiahnutie tokenizéra a základného modelu BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/Users/annahavryliak/IdeaProjects/ZS2/pythonProject/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
#Vytvorenie triedy pre dátasety
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tweet = self.data.iloc[idx]['cleaned_tweet']
        label = self.data.iloc[idx]['label']

        #Tokenizácia textu
        encoding = self.tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'tweet': tweet,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [6]:
#Previesť label (real -> 0, fake -> 1)
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])
val_data['label'] = label_encoder.transform(val_data['label'])

In [7]:
#Vytváranie datasetov a súborov loadov
train_dataset = TweetDataset(train_data, tokenizer, MAX_LEN)
val_dataset = TweetDataset(val_data, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [8]:
#Vytváranie modelu
class BERTClassifier(torch.nn.Module):
    def __init__(self, bert_model, dropout=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(768, 2)  # 2 класи: 0 - real, 1 - fake

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        return linear_output

model = BERTClassifier(bert_model)

In [9]:
#Definícia zariadenia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [10]:
#Funkcia straty a optimalizátor
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

/Users/annahavryliak/IdeaProjects/ZS2/pythonProject/.venv/lib/python3.12/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
#Školenie modelu
def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader), correct_predictions.double() / len(data_loader.dataset)

In [12]:
#Hodnotenie modelu
def eval_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_loss += loss.item()

    return total_loss / len(data_loader), correct_predictions.double() / len(data_loader.dataset)

In [13]:
#Tréning pre 10 epoch
best_accuracy = 0
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f'Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}')

    val_loss, val_acc = eval_model(model, val_loader, criterion, device)
    print(f'Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}')

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_bert_model.pth')
        best_accuracy = val_acc

Epoch 1/10
Train loss: 0.2583, Train accuracy: 0.8975
Validation loss: 0.1931, Validation accuracy: 0.9262
Epoch 2/10
Train loss: 0.0924, Train accuracy: 0.9698
Validation loss: 0.1738, Validation accuracy: 0.9435
Epoch 3/10
Train loss: 0.0436, Train accuracy: 0.9877
Validation loss: 0.1673, Validation accuracy: 0.9528
Epoch 4/10
Train loss: 0.0260, Train accuracy: 0.9917
Validation loss: 0.1851, Validation accuracy: 0.9533
Epoch 5/10
Train loss: 0.0083, Train accuracy: 0.9978
Validation loss: 0.2210, Validation accuracy: 0.9547
Epoch 6/10
Train loss: 0.0151, Train accuracy: 0.9953
Validation loss: 0.2374, Validation accuracy: 0.9449
Epoch 7/10
Train loss: 0.0105, Train accuracy: 0.9964
Validation loss: 0.2033, Validation accuracy: 0.9537
Epoch 8/10
Train loss: 0.0085, Train accuracy: 0.9977
Validation loss: 0.2291, Validation accuracy: 0.9486
Epoch 9/10
Train loss: 0.0073, Train accuracy: 0.9980
Validation loss: 0.2366, Validation accuracy: 0.9542
Epoch 10/10
Train loss: 0.0100, Train